In [10]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import re


In [11]:
# get csv file
data_path = "data/raw/data/beetle.csv"
df = pd.read_csv(data_path)


In [12]:
def preprocess_text(text):
    # Remove extra white spaces
    text = re.sub(r'\s+', ' ', text)

    # Convert to lowercase
    text = text.lower()

    return text


In [13]:
# Preprocess student and reference answers in the dataset
df['student_answer'] = df['student_answer'].apply(preprocess_text)
df['reference_answer'] = df['reference_answer'].apply(preprocess_text)


In [14]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def encode_sentence_pair(student_answer, reference_answer, max_length=512):
    return tokenizer.encode_plus(
        student_answer,
        text_pair=reference_answer,
        max_length=max_length,
        pad_to_max_length=True,
        truncation=True,
        return_attention_mask=True,
        return_tensors="pt",
    )


In [15]:
df.head(1)

,row_id,question,question_id,student_answer,reference_answer,assigned_points,max_points,domain
0,0,What role does the path play in determining wh...,0,if that switch is with the path between that b...,if a bulb and a switch are in the same path th...,1,1,NaN


In [22]:
class ASAGDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        student_answer = row["student_answer"]
        reference_answer = row["reference_answer"]
        percentage_of_correctness = float(row["assigned_points"]) / float(row["max_points"])

        encoded = encode_sentence_pair(student_answer, reference_answer)
        input_ids = encoded["input_ids"].squeeze()
        attention_mask = encoded["attention_mask"].squeeze()
        token_type_ids = encoded.get("token_type_ids", None)
        if token_type_ids is not None:
            token_type_ids = token_type_ids.squeeze()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "token_type_ids": token_type_ids,
            "percentage_of_correctness": torch.tensor(percentage_of_correctness, dtype=torch.float32),
            "assigned_points": torch.tensor(row["assigned_points"], dtype=torch.float32),
            "max_points": torch.tensor(row["max_points"], dtype=torch.float32),
        }


In [23]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

train_dataset = ASAGDataset(train_df)
val_dataset = ASAGDataset(val_df)

batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [18]:
device = "mps" if getattr(torch,'has_mps',False) \
    else "gpu" if torch.cuda.is_available() else "cpu"

model = BertForSequenceClassification.from_pretrained(model_name, num_labels=1)
model.to(device)

num_epochs = 3
optimizer = AdamW(model.parameters(), lr=2e-5)

total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch in train_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        percentage_of_correctness = batch["percentage_of_correctness"].to(device)

        model.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=percentage_of_correctness.unsqueeze(1))
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

        train_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs} - Train loss: {train_loss / len(train_dataloader)}")

# Evaluation loop
model.eval()
predictions = []
ground_truth = []

with torch.no_grad():
    for batch in val_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        percentage_of_correctness = batch["percentage_of_correctness"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits.squeeze().detach().cpu()

        predictions.extend(logits.tolist())
        ground_truth.extend(percentage_of_correctness.cpu().tolist())

# Calculate the mean squared error
mse = mean_squared_error(ground_truth, predictions)
print(f"Mean Squared Error: {mse}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch 1/3 - Train loss: 0.1838835891550041
Epoch 2/3 - Train loss: 0.12693635819429178
Epoch 3/3 - Train loss: 0.10105955904051436
Mean Squared Error: 0.11197731473622943


In [24]:
# Evaluation loop
model.eval()
predictions = []
ground_truth = []
num_correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in val_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        percentage_of_correctness = batch["percentage_of_correctness"].to(device)
        max_points = batch["max_points"].to(device)
        assigned_points = batch["assigned_points"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits.squeeze().detach()

        # Multiply the predicted percentage of correctness by max_points and round to the nearest integer
        rounded_predictions = torch.round(logits * max_points)

        # Compare the rounded_predictions to the assigned_points and count the number of correct predictions
        num_correct_predictions += torch.sum(rounded_predictions == assigned_points).item()
        total_predictions += assigned_points.size(0)

        predictions.extend(logits.tolist())
        ground_truth.extend(percentage_of_correctness.cpu().tolist())

# Calculate the mean squared error
mse = mean_squared_error(ground_truth, predictions)
print(f"Mean Squared Error: {mse}")

# Calculate the accuracy
accuracy = num_correct_predictions / total_predictions
print(f"Accuracy: {accuracy * 100:.2f}%")


/Users/olaf/opt/miniconda3/envs/torch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Mean Squared Error: 0.11197731473622943
Accuracy: 84.52%


In [4]:
from transformers import BertTokenizer

# Load pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Some text
text = "Replace me by any text you'd like."

# Tokenize input for BERT (adding special tokens and creating attention masks)
inputs = tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors='pt')

print(inputs)


{'input_ids': tensor([[ 101, 5672, 2033, 2011, 2151, 3793, 2017, 1005, 1040, 2066, 1012,  102,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0,